In [1]:
# Update sklearn to prevent version mismatches
#!pip install sklearn --upgrade  !! This has been done

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1320 sha256=c3e40cfb58143f1baca7e5f60af88095fb640fce30c1cc9f3cb6719f57a19329
  Stored in directory: c:\users\gshre\appdata\local\pip\cache\wheels\46\ef\c3\157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built sklearn


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
#!pip install joblib  !! This has been done

In [1]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [2]:
df = pd.read_csv("exoplanet_data.csv")
print(df.shape)
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
# remove false positive records and other records.....can't do this
#df = df[~(df["koi_disposition"].str.match("CANDIDATE"))]
print(df.shape)
df.head()

(6991, 41)
(6991, 41)


,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [3]:
# Set features. This will also be used as your x values.
df['koi_fpflag_nt'] = df['koi_fpflag_nt'].astype('category')
df['koi_fpflag_ss'] = df['koi_fpflag_ss'].astype('category')
df['koi_fpflag_co'] = df['koi_fpflag_co'].astype('category')
df['koi_fpflag_ec'] = df['koi_fpflag_ec'].astype('category')

In [4]:
# Copy of all features
selected_features = df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec',
                        'koi_period', 'koi_period_err1', 'koi_period_err2', 
                        'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 
                        'koi_impact', 'koi_impact_err1','koi_impact_err2',
                        'koi_duration', 'koi_duration_err1', 'koi_duration_err2', 
                        'koi_depth', 'koi_depth_err1', 'koi_depth_err2',
                        'koi_prad', 'koi_prad_err1', 'koi_prad_err2', 
                        'koi_teq', 
                        'koi_insol',  'koi_insol_err1', 'koi_insol_err2',
                        'koi_model_snr', 
                        'koi_tce_plnt_num',
                        'koi_steff', 'koi_steff_err1', 'koi_steff_err2', 
                        'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2', 
                        'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 
                        'ra', 
                        'dec', 
                        'koi_kepmag' 
                       ]]

selected_features.describe()

,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
count,6991.000000,6.991000e+03,6.991000e+03,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,...,6991.000000,6991.000000,6991.000000,6991.00000,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000,6991.000000
mean,56.191248,1.851122e-03,-1.851122e-03,164.488820,0.009340,-0.009340,0.730631,1.945368,-0.337243,5.549368,...,-161.206980,4.305049,0.121091,-0.14048,1.740749,0.352710,-0.388568,292.082406,43.812143,14.271508
std,117.570962,7.184503e-03,7.184503e-03,67.020475,0.021989,0.021989,3.346956,9.401850,1.349036,6.425205,...,71.448481,0.439238,0.132048,0.08199,5.903415,0.839017,1.907797,4.762908,3.606167,1.350802
min,0.259820,1.100000e-08,-1.568000e-01,120.515914,0.000009,-0.569000,0.000000,0.000000,-59.320000,0.167000,...,-1733.000000,0.047000,0.000000,-1.00700,0.109000,0.000000,-103.825000,279.856080,36.577381,6.966000
25%,2.620126,5.005000e-06,-2.401000e-04,132.683917,0.001145,-0.010000,0.198000,0.039900,-0.446000,2.446000,...,-197.000000,4.209000,0.044000,-0.19500,0.829000,0.128000,-0.252000,288.704730,40.797760,13.455000
50%,8.947426,3.300000e-05,-3.300000e-05,136.739230,0.003990,-0.003990,0.539000,0.189800,-0.204000,3.793000,...,-159.000000,4.436000,0.070000,-0.12700,0.999000,0.248000,-0.111000,292.314760,43.679661,14.534000
75%,34.282605,2.401000e-04,-5.005000e-06,169.937005,0.010000,-0.001145,0.891000,0.379000,-0.044000,6.197500,...,-112.000000,4.543000,0.149000,-0.08800,1.357000,0.357000,-0.069000,295.888550,46.693659,15.322000
max,1071.232624,1.568000e-01,-1.100000e-08,1472.522306,0.569000,-0.000009,100.806000,85.540000,0.000000,138.540000,...,0.000000,5.364000,1.472000,0.00000,180.013000,25.956000,0.000000,301.720760,52.336010,19.065000


# Remove err values
selected_features = df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co', 'koi_fpflag_ec',
                        'koi_period', 
                        'koi_time0bk',
                        'koi_impact', 
                        'koi_duration', 
                        'koi_depth',
                        'koi_prad', 
                        'koi_teq', 
                        'koi_insol', 
                        'koi_model_snr', 
                        'koi_tce_plnt_num',
                        'koi_steff', 
                        'koi_slogg',
                        'koi_srad',
                        'ra', 
                        'dec', 
                        'koi_kepmag' 
                       ]]

selected_features.describe()

# Create a Train Test Split

Use `koi_disposition` for the y values

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(selected_features, df["koi_disposition"], random_state=23)

In [6]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
243,0,0,0,0,5.714624,0.000033,-0.000033,132.638660,0.004320,-0.004320,...,-80,4.365,0.132,-0.096,1.013,0.141,-0.141,296.06424,45.976841,15.596
5885,0,1,0,0,27.308150,0.000004,-0.000004,143.504361,0.000113,-0.000113,...,-163,4.574,0.064,-0.096,0.739,0.127,-0.068,291.37350,51.153381,15.529
3093,0,0,0,0,91.957230,0.005680,-0.005680,183.024800,0.065700,-0.065700,...,-79,4.114,0.182,-0.098,1.528,0.238,-0.327,289.90417,38.651371,14.876
2176,0,0,0,0,31.517795,0.000094,-0.000094,185.153300,0.002230,-0.002230,...,-91,4.144,0.195,-0.105,1.400,0.211,-0.316,297.27798,48.320332,12.848
974,0,0,0,0,32.773763,0.000179,-0.000179,151.443520,0.004630,-0.004630,...,-76,4.076,0.203,-0.087,1.534,0.240,-0.360,290.88232,43.471760,14.758


In [7]:
y_train.head()

243          CONFIRMED
5885    FALSE POSITIVE
3093         CANDIDATE
2176         CONFIRMED
974          CANDIDATE
Name: koi_disposition, dtype: object

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [8]:
# Scale your data
from sklearn.preprocessing import MinMaxScaler

X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
print(X_train_scaled)

[[0.         0.         0.         ... 0.74129418 0.60087477 0.71328209]
 [0.         1.         0.         ... 0.52675914 0.93179296 0.70774444]
 [0.         0.         0.         ... 0.45955806 0.13258296 0.65377304]
 ...
 [0.         0.         0.         ... 0.27782753 0.16114342 0.63277957]
 [0.         1.         1.         ... 0.84952764 0.48680493 0.52227457]
 [1.         0.         0.         ... 0.81032103 0.55334272 0.70427308]]


In [9]:
# HotKey Encode the data
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)
print(encoded_y_train)

[1 2 0 ... 1 2 2]


# Train the Model



In [10]:
# Create a Logistic Regression Classifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
classifier =  RandomForestClassifier() 

# fit the model with the training data
classifier.fit(X_train_scaled, encoded_y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [11]:
print(f"Training Data Score: {classifier.score(X_train_scaled, encoded_y_train)}")
print(f"Testing Data Score: {classifier.score(X_test_scaled, encoded_y_test)}")

Training Data Score: 1.0
Testing Data Score: 0.9073226544622426


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [29]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV

# (max_depth=5, n_estimators=10, max_features=1

param_grid = {'max_depth': [1, 2, 5, 10],
              'n_estimators': [1, 5, 10, 15],
              'n_jobs': [1, 5, 10],
              'max_features': [1, 5, 10, 20, 25]}
grid = GridSearchCV(classifier, param_grid, verbose=3)
print(grid.estimator.get_params().keys())

dict_keys(['bootstrap', 'ccp_alpha', 'class_weight', 'criterion', 'max_depth', 'max_features', 'max_leaf_nodes', 'max_samples', 'min_impurity_decrease', 'min_impurity_split', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_jobs', 'oob_score', 'random_state', 'verbose', 'warm_start'])


# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV

param_grid = {'C': [1, 5, 10, 50],
              'kernel': ['linear'],
              'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [30]:
# Train the model with GridSearch
grid.fit(X_train_scaled, encoded_y_train)

Fitting 5 folds for each of 360 candidates, totalling 1800 fits
[CV] max_depth=1, max_features=1, n_estimators=1, n_jobs=1 ...........
[CV]  max_depth=1, max_features=1, n_estimators=1, n_jobs=1, score=0.504, total=   0.0s
[CV] max_depth=1, max_features=1, n_estimators=1, n_jobs=1 ...........
[CV]  max_depth=1, max_features=1, n_estimators=1, n_jobs=1, score=0.504, total=   0.0s
[CV] max_depth=1, max_features=1, n_estimators=1, n_jobs=1 ...........
[CV]  max_depth=1, max_features=1, n_estimators=1, n_jobs=1, score=0.534, total=   0.0s
[CV] max_depth=1, max_features=1, n_estimators=1, n_jobs=1 ...........
[CV]  max_depth=1, max_features=1, n_estimators=1, n_jobs=1, score=0.504, total=   0.0s
[CV] max_depth=1, max_features=1, n_estimators=1, n_jobs=1 ...........
[CV]  max_depth=1, max_features=1, n_estimators=1, n_jobs=1, score=0.504, total=   0.0s
[CV] max_depth=1, max_features=1, n_estimators=1, n_jobs=5 ...........


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s


[CV]  max_depth=1, max_features=1, n_estimators=1, n_jobs=5, score=0.504, total=   0.5s
[CV] max_depth=1, max_features=1, n_estimators=1, n_jobs=5 ...........
[CV]  max_depth=1, max_features=1, n_estimators=1, n_jobs=5, score=0.504, total=   0.3s
[CV] max_depth=1, max_features=1, n_estimators=1, n_jobs=5 ...........
[CV]  max_depth=1, max_features=1, n_estimators=1, n_jobs=5, score=0.504, total=   0.3s
[CV] max_depth=1, max_features=1, n_estimators=1, n_jobs=5 ...........
[CV]  max_depth=1, max_features=1, n_estimators=1, n_jobs=5, score=0.504, total=   0.3s
[CV] max_depth=1, max_features=1, n_estimators=1, n_jobs=5 ...........
[CV]  max_depth=1, max_features=1, n_estimators=1, n_jobs=5, score=0.504, total=   0.3s
[CV] max_depth=1, max_features=1, n_estimators=1, n_jobs=10 ..........
[CV]  max_depth=1, max_features=1, n_estimators=1, n_jobs=10, score=0.504, total=   0.0s
[CV] max_depth=1, max_features=1, n_estimators=1, n_jobs=10 ..........
[CV]  max_depth=1, max_features=1, n_estimato

[CV]  max_depth=1, max_features=1, n_estimators=15, n_jobs=10, score=0.504, total=   0.6s
[CV] max_depth=1, max_features=1, n_estimators=15, n_jobs=10 .........
[CV]  max_depth=1, max_features=1, n_estimators=15, n_jobs=10, score=0.504, total=   0.2s
[CV] max_depth=1, max_features=1, n_estimators=15, n_jobs=10 .........
[CV]  max_depth=1, max_features=1, n_estimators=15, n_jobs=10, score=0.504, total=   0.2s
[CV] max_depth=1, max_features=2, n_estimators=1, n_jobs=1 ...........
[CV]  max_depth=1, max_features=2, n_estimators=1, n_jobs=1, score=0.504, total=   0.0s
[CV] max_depth=1, max_features=2, n_estimators=1, n_jobs=1 ...........
[CV]  max_depth=1, max_features=2, n_estimators=1, n_jobs=1, score=0.504, total=   0.0s
[CV] max_depth=1, max_features=2, n_estimators=1, n_jobs=1 ...........
[CV]  max_depth=1, max_features=2, n_estimators=1, n_jobs=1, score=0.504, total=   0.0s
[CV] max_depth=1, max_features=2, n_estimators=1, n_jobs=1 ...........
[CV]  max_depth=1, max_features=2, n_est

[CV]  max_depth=1, max_features=2, n_estimators=15, n_jobs=5, score=0.504, total=   0.4s
[CV] max_depth=1, max_features=2, n_estimators=15, n_jobs=5 ..........
[CV]  max_depth=1, max_features=2, n_estimators=15, n_jobs=5, score=0.504, total=   0.2s
[CV] max_depth=1, max_features=2, n_estimators=15, n_jobs=5 ..........
[CV]  max_depth=1, max_features=2, n_estimators=15, n_jobs=5, score=0.504, total=   0.2s
[CV] max_depth=1, max_features=2, n_estimators=15, n_jobs=5 ..........
[CV]  max_depth=1, max_features=2, n_estimators=15, n_jobs=5, score=0.504, total=   0.1s
[CV] max_depth=1, max_features=2, n_estimators=15, n_jobs=5 ..........
[CV]  max_depth=1, max_features=2, n_estimators=15, n_jobs=5, score=0.625, total=   0.2s
[CV] max_depth=1, max_features=2, n_estimators=15, n_jobs=10 .........
[CV]  max_depth=1, max_features=2, n_estimators=15, n_jobs=10, score=0.504, total=   0.2s
[CV] max_depth=1, max_features=2, n_estimators=15, n_jobs=10 .........
[CV]  max_depth=1, max_features=2, n_es

...

[CV]  max_depth=10, max_features=20, n_estimators=1, n_jobs=10, score=0.867, total=   0.1s
[CV] max_depth=10, max_features=20, n_estimators=1, n_jobs=10 ........
[CV]  max_depth=10, max_features=20, n_estimators=1, n_jobs=10, score=0.876, total=   0.0s
[CV] max_depth=10, max_features=20, n_estimators=1, n_jobs=10 ........
[CV]  max_depth=10, max_features=20, n_estimators=1, n_jobs=10, score=0.856, total=   0.0s
[CV] max_depth=10, max_features=20, n_estimators=1, n_jobs=10 ........
[CV]  max_depth=10, max_features=20, n_estimators=1, n_jobs=10, score=0.871, total=   0.0s
[CV] max_depth=10, max_features=20, n_estimators=1, n_jobs=10 ........
[CV]  max_depth=10, max_features=20, n_estimators=1, n_jobs=10, score=0.829, total=   0.0s
[CV] max_depth=10, max_features=20, n_estimators=5, n_jobs=1 .........
[CV]  max_depth=10, max_features=20, n_estimators=5, n_jobs=1, score=0.892, total=   0.1s
[CV] max_depth=10, max_features=20, n_estimators=5, n_jobs=1 .........
[CV]  max_depth=10, max_featu

[CV]  max_depth=10, max_features=25, n_estimators=1, n_jobs=1, score=0.869, total=   0.0s
[CV] max_depth=10, max_features=25, n_estimators=1, n_jobs=5 .........
[CV]  max_depth=10, max_features=25, n_estimators=1, n_jobs=5, score=0.870, total=   0.3s
[CV] max_depth=10, max_features=25, n_estimators=1, n_jobs=5 .........
[CV]  max_depth=10, max_features=25, n_estimators=1, n_jobs=5, score=0.869, total=   0.0s
[CV] max_depth=10, max_features=25, n_estimators=1, n_jobs=5 .........
[CV]  max_depth=10, max_features=25, n_estimators=1, n_jobs=5, score=0.863, total=   0.0s
[CV] max_depth=10, max_features=25, n_estimators=1, n_jobs=5 .........
[CV]  max_depth=10, max_features=25, n_estimators=1, n_jobs=5, score=0.865, total=   0.0s
[CV] max_depth=10, max_features=25, n_estimators=1, n_jobs=5 .........
[CV]  max_depth=10, max_features=25, n_estimators=1, n_jobs=5, score=0.852, total=   0.0s
[CV] max_depth=10, max_features=25, n_estimators=1, n_jobs=10 ........
[CV]  max_depth=10, max_features=2

[CV]  max_depth=10, max_features=25, n_estimators=15, n_jobs=10, score=0.888, total=   0.4s
[CV] max_depth=10, max_features=25, n_estimators=15, n_jobs=10 .......
[CV]  max_depth=10, max_features=25, n_estimators=15, n_jobs=10, score=0.900, total=   0.6s
[CV] max_depth=10, max_features=25, n_estimators=15, n_jobs=10 .......
[CV]  max_depth=10, max_features=25, n_estimators=15, n_jobs=10, score=0.903, total=   0.5s
[CV] max_depth=10, max_features=25, n_estimators=15, n_jobs=10 .......
[CV]  max_depth=10, max_features=25, n_estimators=15, n_jobs=10, score=0.890, total=   0.3s
[CV] max_depth=10, max_features=25, n_estimators=15, n_jobs=10 .......
[CV]  max_depth=10, max_features=25, n_estimators=15, n_jobs=10, score=0.886, total=   0.3s


[Parallel(n_jobs=1)]: Done 1800 out of 1800 | elapsed:  4.7min finished


GridSearchCV(cv=None, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              ra

In [33]:
print(grid.best_params_)
print(grid.best_score_)

{'max_depth': 10, 'max_features': 20, 'n_estimators': 10, 'n_jobs': 1}
0.8943353903026511


# Apply best parameters

In [35]:
classifier =  RandomForestClassifier(max_depth=10, max_features=20, n_jobs=1, n_estimators=10) 

# fit the model with the training data
classifier.fit(X_train_scaled, encoded_y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features=20,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [36]:
print(f"Training Data Score: {classifier.score(X_train_scaled, encoded_y_train)}")
print(f"Testing Data Score: {classifier.score(X_test_scaled, encoded_y_test)}")

Training Data Score: 0.9303833683005913
Testing Data Score: 0.8958810068649885


# Save the Model

In [37]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'random_forest_classifier_model.sav'
joblib.dump(classifier, filename)

['random_forest_classifier_model.sav']